In [5]:
import numpy as np # liniear algebra
import pandas as pd # csv I/O
import os # file operations
import h5py # handle data that doesn't fit in memory

TRAIN_DIR = r'D:\LICENTA\train_photos'
TEST_DIR = r'D:\LICENTA\test_photos'
IMG_SIZE = 299

DATA_DIR = r'D:\LICENTA\processed_data\size_{size1}x{size2}'.format(size1=IMG_SIZE, size2=IMG_SIZE)

train_labels = pd.read_csv('train.csv')

In [ ]:
%%time
# VGG16
from keras.applications.vgg16 import VGG16 # predefined CNN Model
from keras.preprocessing import image # get the array representation of the image
from keras.applications.vgg16 import preprocess_input # get the array in a format compatible with the model

# takes around 45 seconds
model = VGG16(weights='imagenet', include_top=False, pooling='max')

model.summary()

In [ ]:
%%time
# RESNET50
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input

model = ResNet50(weights='imagenet', include_top=False, pooling='max')

model.summary()

In [ ]:
%%time
# INCEPTION V3
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.applications.inception_v3 import preprocess_input

model = InceptionV3(weights='imagenet', include_top=False, pooling='max')

model.summary()

In [6]:
%%time
# XCEPTION
from keras.applications.xception import Xception
from keras.preprocessing import image
from keras.applications.xception import preprocess_input

model = Xception(weights='imagenet', include_top=False, pooling='max')

model.summary()

Using TensorFlow backend.


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, None, None, 3) 0                                            
____________________________________________________________________________________________________
block1_conv1 (Conv2D)            (None, None, None, 32 864         input_1[0][0]                    
____________________________________________________________________________________________________
block1_conv1_bn (BatchNormalizat (None, None, None, 32 128         block1_conv1[0][0]               
____________________________________________________________________________________________________
block1_conv1_act (Activation)    (None, None, None, 32 0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

____________________________________________________________________________________________________
block13_pool (MaxPooling2D)      (None, None, None, 10 0           block13_sepconv2_bn[0][0]        
____________________________________________________________________________________________________
batch_normalization_4 (BatchNorm (None, None, None, 10 4096        conv2d_4[0][0]                   
____________________________________________________________________________________________________
add_12 (Add)                     (None, None, None, 10 0           block13_pool[0][0]               
                                                                   batch_normalization_4[0][0]      
____________________________________________________________________________________________________
block14_sepconv1 (SeparableConv2 (None, None, None, 15 1582080     add_12[0][0]                     
___________________________________________________________________________________________

In [7]:
MODEL_NAME = 'xception'
FEATURE_VECTOR_SIZE = 2048

In [8]:
def extract_features(imgs_path, img_size):
    
    features = []
    
    for path in imgs_path:
        img = image.load_img(path, target_size=(IMG_SIZE, IMG_SIZE))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = preprocess_input(img_array)
        features.append(model.predict(img_array).reshape(FEATURE_VECTOR_SIZE,))
    
    return np.array(features)

In [ ]:
# CREATE TRAIN DATA FILE

with h5py.File(DATA_DIR + r'\train_images_{name}_features.h5'.format(name=MODEL_NAME), 'w') as f:
    img_names = f.create_dataset('photo_id', (0,), maxshape=(None,), dtype='|S54')
    feature = f.create_dataset('feature', (0, FEATURE_VECTOR_SIZE), maxshape=(None, FEATURE_VECTOR_SIZE))

In [9]:
# TRAIN PHOTOS

import time
from datetime import datetime

train_photos = pd.read_csv('train_photo_to_biz_ids.csv')
train_images = [os.path.join(TRAIN_DIR, str(x) + '.jpg') for x in train_photos['photo_id']]

num_train = len(train_images)
print('Number of train images:', num_train, 'started at', str(datetime.now()))
batch_size = 4000
num_done = 0
t = time.time()

for i in range(120000, num_train, batch_size):
    images = train_images[i:min(i + batch_size, num_train)]
    features = extract_features(images, IMG_SIZE)
    num_done = i + features.shape[0]
    
    with h5py.File(DATA_DIR + r'\train_images_{name}_features.h5'.format(name=MODEL_NAME), 'r+') as f:
        f['photo_id'].resize((num_done,))
        f['photo_id'][i:num_done] = np.array(images).astype('|S54')
        f['feature'].resize((num_done, features.shape[1]))
        f['feature'][i:num_done,:] = features
    
    if num_done % batch_size == 0 or num_done == num_train:
        print('Train images proccesed:', num_done, 'time passed: ', '{0:.1f}'.format(time.time() - t), 'sec hour:', str(datetime.now()))


Number of train images: 234842 started at 2017-12-06 07:20:42.572974
Train images proccesed: 124000 time passed:  814.5 sec hour: 2017-12-06 07:34:17.070617
Train images proccesed: 128000 time passed:  1602.3 sec hour: 2017-12-06 07:47:24.856612
Train images proccesed: 132000 time passed:  2376.8 sec hour: 2017-12-06 08:00:19.390952
Train images proccesed: 136000 time passed:  3141.7 sec hour: 2017-12-06 08:13:04.268790
Train images proccesed: 140000 time passed:  3899.3 sec hour: 2017-12-06 08:25:41.918994
Train images proccesed: 144000 time passed:  4647.4 sec hour: 2017-12-06 08:38:09.990604
Train images proccesed: 148000 time passed:  5398.1 sec hour: 2017-12-06 08:50:40.680731
Train images proccesed: 152000 time passed:  6142.9 sec hour: 2017-12-06 09:03:05.504415
Train images proccesed: 156000 time passed:  6884.6 sec hour: 2017-12-06 09:15:27.147944
Train images proccesed: 160000 time passed:  7621.3 sec hour: 2017-12-06 09:27:43.872021
Train images proccesed: 164000 time passed

In [10]:
# Check the file content

with h5py.File(DATA_DIR + r'\train_images_{name}_features.h5'.format(name=MODEL_NAME),'r') as f:
    print(r'\train_images_{name}_features.h5'.format(name=MODEL_NAME))
    for key in f.keys():
        print(key, f[key].shape)

    print("\nA photo:", f['photo_id'][0])
    print("Its feature vector (first 10-dim): ", f['feature'][0][0:10], " ...")

\train_images_xception_features.h5
feature (234842, 2048)
photo_id (234842,)

A photo: b'D:\\LICENTA\\train_photos\\204149.jpg'
Its feature vector (first 10-dim):  [ 0.26662534  1.86572039  2.70066929  0.          1.86418772  0.38295555
  0.          0.6968208   0.44267178  0.        ]  ...


In [ ]:
# TEST PHOTOS

with h5py.File(DATA_DIR + r'\test_images_{name}_features.h5'.format(name=MODEL_NAME), 'w') as f:
    img_names = f.create_dataset('photo_id', (0,), maxshape=(None,), dtype='|S54')
    feature = f.create_dataset('feature', (0, FEATURE_VECTOR_SIZE), maxshape=(None, FEATURE_VECTOR_SIZE))
    
test_photos = pd.read_csv('test_photo_to_biz.csv')
test_images = [os.path.join(TEST_DIR, str(x) + '.jpg') for x in test_photos['photo_id'].unique()]

num_test = len(test_images)
print('Number of test images: ', num_test)
batch_size = 4000

for i in range(0, num_test, batch_size):
    images = test_images[i:min(i + batch_size, num_test)]
    features = extract_features(images, IMG_SIZE)
    num_done = i + features.shape[0]
    
    with h5py.File(DATA_DIR + r'\test_images_{name}_features.h5'.format(name=MODEL_NAME), 'r+') as f:
        f['photo_id'].resize((num_done,))
        f['photo_id'][i:num_done] = np.array(images).astype('|S54')
        f['feature'].resize((num_done, features.shape[1]))
        f['feature'][i:num_done,:] = features
    
    if num_done % batch_size == 0 or num_done == num_train:
        print('Test images proccesed:', num_done)

In [ ]:
# Check the file content

with h5py.File(DATA_DIR + r'\test_images_{name}_features.h5'.format(name=MODEL_NAME),'r') as f:
    for key in f.keys():
        print(key, f[key].shape)

    print "\nA photo:", f['photo_id'][0]
    print("feature vector: (first 10-dim)", f['feature'][0][0:10], " ...")